In [4]:
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
import sys

In [5]:
credits = pd.read_csv('../backend/tmdb_5000_credits.csv')
movies = pd.read_csv("../backend/tmdb_5000_movies.csv")

In [6]:
#merging both files
movies=movies.rename(index=str,columns={'id':'movie_id'})
movies = movies.merge(credits, on='movie_id')

In [7]:
movies['overview'] = movies['overview'].fillna('')

In [8]:
movies['overview']

0       In the 22nd century, a paraplegic Marine is di...
1       Captain Barbossa, long believed to be dead, ha...
2       A cryptic message from Bond’s past sends him o...
3       Following the death of District Attorney Harve...
4       John Carter is a war-weary, former military ca...
                              ...                        
4798    El Mariachi just wants to play his guitar and ...
4799    A newlywed couple's honeymoon is upended by th...
4800    "Signed, Sealed, Delivered" introduces a dedic...
4801    When ambitious New York attorney Sam is sent t...
4802    Ever since the second grade when he first saw ...
Name: overview, Length: 4803, dtype: object

In [9]:
countvectorizer = CountVectorizer()
countvectorizer_matrix = countvectorizer.fit_transform(movies['overview'])

In [10]:
print(countvectorizer_matrix.shape)

(4803, 21262)


In [90]:
kernel = linear_kernel(countvectorizer_matrix, countvectorizer_matrix)

indexes = pd.Series(movies.index,index=movies['original_title'])
print(kernel.shape)

(4803, 4803)
[[ 28.  12.  10. ...  18.  11.   9.]
 [ 12.  46.  17. ...  23.  21.  13.]
 [ 10.  17.  51. ...  25.  15.  14.]
 ...
 [ 18.  23.  25. ... 112.  32.  19.]
 [ 11.  21.  15. ...  32.  84.  22.]
 [  9.  13.  14. ...  19.  22.  80.]]


In [12]:
print(indexes)

original_title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4798
Newlyweds                                   4799
Signed, Sealed, Delivered                   4800
Shanghai Calling                            4801
My Date with Drew                           4802
Length: 4803, dtype: int64


In [13]:
#title = sys.argv[1]
title="Deadpool"

In [14]:
f=0
idx=-1
for i in indexes.iteritems():
    if(i[0]==title):
        f=1
        #print("found")
        idx=i[1]
        break
if f==0:
    res="Not Found"
    print(res)
    sys.stdout.flush()
    exit(0)
print(idx)

788


In [59]:
#Find similarity
similarity = list(enumerate(kernel[idx]))
len(similarity)
#print(similarity)

4803

In [60]:
#Sort by descnding to get top matches
similarity = sorted(similarity, key=lambda x: x[1], reverse=True)
#similarity=similarity.sort(key=lambda x:x[1],reverse=True)
similarity = similarity[0:6]
print(similarity)

[(274, 103.0), (2647, 101.0), (3432, 101.0), (4123, 96.0), (847, 93.0), (4562, 90.0)]


In [67]:
movie_indexes = []
movie_scores=[]
for row in similarity:
    movie_indexes.append(row[0])
    movie_scores.append(row[1])

In [68]:
print(movie_indexes)

[274, 2647, 3432, 4123, 847, 4562]


In [78]:
x=movies['original_title'].iloc[movie_indexes]
print(np.shape(x))
print(x)

(6,)
274                                    Gladiator
2647                     The Midnight Meat Train
3432    The Work and the Glory II: American Zion
4123                              Roadside Romeo
847                                     Semi-Pro
4562                    The Little Ponderosa Zoo
Name: original_title, dtype: object


In [86]:
res=[]
for i in range(len(x)):
    res.append(x.iloc[i]+":"+str(movie_scores[i]))
    #res.append(movie_scores[i])
print(res)
#sys.stdout.flush()

['Gladiator:103.0', 'The Midnight Meat Train:101.0', 'The Work and the Glory II: American Zion:101.0', 'Roadside Romeo:96.0', 'Semi-Pro:93.0', 'The Little Ponderosa Zoo:90.0']
